In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

## Setup e Importação

In [ ]:
# Caminho do arquivo CSV no Google Drive
csv_file = os.getcwd() +"\\dados\\dados_tuberculose_2013_2023_limpo.csv"
xlsx_file = os.getcwd() +"\\dados\\pop_residente_municipios.xlsx"

df_raw = pd.read_csv(csv_file, encoding='latin1', low_memory=False)
df_munic = pd.read_excel(xlsx_file)

In [6]:
dummy_var = ('CS_SEXO')

## Juntando as Tabelas

In [43]:
df_munic_tidy = df_munic.melt(id_vars=['MUNICIPIO', 'ID_MUNIC', 'NOME_MUNIC'], var_name='ANO', value_name='POP_RESIDENTE_MUNIC')

In [ ]:
df_merge = df_raw.merge(df_munic_tidy, left_on=['ID_MUNIC_A','NU_ANO'], right_on=['ID_MUNIC', 'ANO'], how='left')

In [ ]:
#Check da quantidade de linhas após merge
df_raw.shape[0] - df_merge.shape[0]

0

In [52]:
df_merge

,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,DT_DIAG,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,ANT_RETRO,BAC_APOS_6,TRANSF,IDADE_ANOS,ABANDONO,MUNICIPIO,ID_MUNIC,NOME_MUNIC,ANO,POP_RESIDENTE_MUNIC
0,2013,42,421010.0,0.0,2013-11-29,4058.0,M,6.0,1.0,1.0,...,0.0,0.0,0.0,58.0,0,410690 CURITIBA,410690.0,CURITIBA,2013,1825074.0
1,2013,42,420460.0,1558.0,2013-08-30,4025.0,M,6.0,1.0,2.0,...,0.0,0.0,0.0,25.0,0,420460 CRICIUMA,420460.0,CRICIUMA,2013,202374.0
2,2013,41,410830.0,1363.0,2013-07-13,4076.0,M,6.0,1.0,9.0,...,2.0,0.0,3.0,76.0,0,410830 FOZ DO IGUACU,410830.0,FOZ DO IGUACU,2013,270746.0
3,2013,31,311860.0,1449.0,2013-06-12,4067.0,M,6.0,2.0,9.0,...,0.0,0.0,0.0,67.0,0,311860 CONTAGEM,311860.0,CONTAGEM,2013,625734.0
4,2013,25,250750.0,1377.0,2013-08-01,4035.0,M,6.0,4.0,1.0,...,0.0,0.0,0.0,35.0,1,250750 JOAO PESSOA,250750.0,JOAO PESSOA,2013,765306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025960,2023,43,431490.0,159.0,2023-12-31,4069.0,M,6.0,1.0,3.0,...,2.0,9.0,9.0,69.0,0,431490 PORTO ALEGRE,431490.0,PORTO ALEGRE,2023,1405685.0
1025961,2023,14,140010.0,9.0,2023-12-31,4023.0,F,5.0,4.0,9.0,...,9.0,9.0,9.0,23.0,0,140010 BOA VISTA,140010.0,BOA VISTA,2023,454980.0
1025962,2023,33,330010.0,9.0,2023-12-31,4042.0,M,6.0,4.0,9.0,...,9.0,9.0,9.0,42.0,0,330010 ANGRA DOS REIS,330010.0,ANGRA DOS REIS,2023,179494.0
1025963,2023,26,261160.0,149.0,2023-12-31,4089.0,M,6.0,4.0,1.0,...,9.0,9.0,9.0,89.0,0,261160 RECIFE,261160.0,RECIFE,2023,1592362.0


## Filtros 

| Nome UF | Código IBGE |
| --- | --- |
|Rondônia|11|
|Acre|12|
|Amazonas|13|
|Roraima|14|
|Pará|15|
|Amapá|16|
|Tocantins|17|
|Maranhão|21|
|Piauí|22|
|Ceará|23|
|Rio Grande do Norte|24|
|Paraíba|25|
|Pernambuco|26|
|Alagoas|27|
|Sergipe|28|
|Bahia|29|
|Minas Gerais|31|
|Espírito Santo|32|
|Rio de Janeiro|33|
|São Paulo|35|
|Paraná|41|
|Santa Catarina|42|
|Rio Grande do Sul|43|
|Mato Grosso do Sul|50|
|Mato Grosso|51|
|Goiás|52|
|Distrito Federal|53|

## Seleção de Variáveis e Tratamento


In [53]:
# 1. Criar o DataFrame com as variáveis selecionadas
variaveis_independentes = [ 'IDADE_ANOS', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_OCUPA_N', 'TRATAMENTO',
    'RAIOX_TORA', 'TESTE_TUBE', 'FORMA', 'AGRAVAIDS', 'AGRAVALCOO', 'AGRAVDIABE',
    'AGRAVDOENC', 'AGRAVOUTRA', 'AGRAVDROGA', 'AGRAVTABAC', 'BACILOSC_E', 'CULTURA_ES', 'CULTURA_OU', 'HIV', 'HISTOPATOL',
    'RIFAMPICIN', 'ISONIAZIDA', 'ETAMBUTOL', 'ESTREPTOMI', 'PIRAZINAMI',
    'ETIONAMIDA','NU_COMU_EX', 'DOENCA_TRA', 'TRATSUP_AT',  
    'TPUNINOT', 'POP_LIBER', 'POP_RUA', 'POP_SAUDE', 'POP_IMIG',
    'BENEF_GOV', 'TEST_MOLEC', 'TEST_SENSI', 'ANT_RETRO', 'TRANSF']

variaveis_local = ['SG_UF_AT','SG_UF_2','ID_MUNIC_A']
variaveis_acompanhamento = ['BACILOSC_1', 'BACILOSC_2', 'BACILOSC_3', 'BACILOSC_4', 'BACILOSC_5', 'BACILOSC_6', 'BAC_APOS_6', 'BACILOS_E2', 'BACILOSC_O', 'SITUA_9_M','SITUA_12_M']
#Estas são variáveis que buscamos em bases externas, representantes das características dos municípios
variaveis_exogenas = ['POP_RESIDENTE_MUNIC']

# Novo DataFrame com as variáveis independentes e alvo
df_modelagem = df_merge[['ABANDONO'] + variaveis_independentes +['ID_MUNIC_A']+['SG_UF']+variaveis_exogenas].copy()

In [55]:
df_modelagem.columns.sort_values()

Index(['ABANDONO', 'AGRAVAIDS', 'AGRAVALCOO', 'AGRAVDIABE', 'AGRAVDOENC',
       'AGRAVDROGA', 'AGRAVOUTRA', 'AGRAVTABAC', 'ANT_RETRO', 'BACILOSC_E',
       'BENEF_GOV', 'CS_ESCOL_N', 'CS_GESTANT', 'CS_RACA', 'CS_SEXO',
       'CULTURA_ES', 'CULTURA_OU', 'DOENCA_TRA', 'ESTREPTOMI', 'ETAMBUTOL',
       'ETIONAMIDA', 'FORMA', 'HISTOPATOL', 'HIV', 'IDADE_ANOS', 'ID_MUNIC_A',
       'ID_OCUPA_N', 'ISONIAZIDA', 'NU_COMU_EX', 'PIRAZINAMI', 'POP_IMIG',
       'POP_LIBER', 'POP_RESIDENTE_MUNIC', 'POP_RUA', 'POP_SAUDE',
       'RAIOX_TORA', 'RIFAMPICIN', 'SG_UF', 'TESTE_TUBE', 'TEST_MOLEC',
       'TEST_SENSI', 'TPUNINOT', 'TRANSF', 'TRATAMENTO', 'TRATSUP_AT'],
      dtype='object')

v4: 'ID_OCUPA_N', 'TESTE_TUBE', 'CULTURA_OU', 'ETIONAMIDA', 'NU_CONTATO','NU_COMU_EX', 'DOENCA_TRA'

<P> 
Comentários: 
<p>

- Retiro campo HIV, senod que já tenho AGRAVAIDS?   
- Variavel ETIONAMIDA está correta? No dicionario está mal definida  
- Variável OUTRAS pode ser retirada? Qual a proporção de nao informados dela? Não estou utilizando a OUTRAS_DES 
    - OUTRAS foi retirado
- Variáveis 'NU_CONTATO','NU_COMU_EX' não seguem regra de negócio. Há registros de maior NU_COMU_EX (numero de contatos examinados) do que NU_CONTATO (numero de contatos existentes). QUal confiar? Alem disso, NU_COMU_EX tem bastante 9. Será que o 9 foi automático?
    - NU_CONTATO foi retirado
- ANT_RETRO não está descrita no dicionário e, se for ANTIRRETROVIRAL, existem mais números do que 0,1,2,9. Vale a pena deixar essa variável?

In [20]:
#Linhas rascunho na exploração

df_raw[['NU_CONTATO','NU_COMU_EX']][df_raw['NU_CONTATO'] - df_raw['NU_COMU_EX'] < 0]
df_raw['ANT_RETRO'].value_counts()/df_raw.shape[0]
df_raw['NU_COMU_EX'].value_counts().sort_values(ascending=False).head(10)

NU_COMU_EX
0.0    437952
1.0    112642
2.0    108887
9.0    105757
3.0     87714
4.0     58082
5.0     36631
6.0     21847
7.0     12790
8.0      9979
Name: count, dtype: int64

In [56]:
df_modelagem.isna().sum().values

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       2195], dtype=int64)

In [57]:
print('CS_SEXO:',df_modelagem['CS_SEXO'].unique())
print('ETIONAMIDA:',df_modelagem['ETIONAMIDA'].unique())

#Checar o significado dessas classificações

CS_SEXO: ['M' 'F' 'I']
ETIONAMIDA: ['2' '0' '1' '9' '/']


In [58]:
#Retirando linhas de categorias com poucas observações
#Possiveis erros ou má interpretações

df_modelagem = df_modelagem[df_modelagem['CS_SEXO'] != 'I']
df_modelagem = df_modelagem[df_modelagem['ETIONAMIDA'] != '/']


In [59]:
#Tratamento Dummies
gender_mapping = {"F":1, "M":0}
df_modelagem[dummy_var] = df_modelagem[dummy_var].map(gender_mapping) 

In [ ]:
df_modelagem['POP_RESIDENTE_MUNIC']

In [60]:
#Convertendo para numerico
convert_to_num = ['ETIONAMIDA', 'POP_RESIDENTE_MUNIC']

for col in convert_to_num:
    df_modelagem[col] = pd.to_numeric(df_modelagem[col])

### ID Município e Informações Externas

IBGE classifica os municípios baseado na quantidade de habitantes e localização: https://www.ibge.gov.br/apps/rural_urbano/#/home  
Comecei com uma classificação simples baseada no IBGE, mas considerar os conglomerados urbanos pode fazer mais sentido.  
_Ideia: clusterizar as cidades. Complicação: clusterizar baseado em que? Como evitar vazamento de dados?_

Classificação atualmente adotada (mais simples):

- Pequeno Porte I: Até 20.000 habitantes
- Pequeno Porte II: De 20.001 a 50.000 habitantes
- Médio Porte: De 50.001 a 100.000 habitantes
- Grande Porte: De 100.001 a 900.000 habitantes
- Metrópole: Acima de 900.000 habitantes 

In [64]:
# Definindo as faixas e os rótulos baseados na classificação do IBGE
faixas = [0, 20000, 50000, 100000, 900000, float('inf')]
rotulos = ['Pequeno I', 'Pequeno II', 'Médio', 'Grande', 'Metrópole']

df_modelagem['porte_municipio'] = pd.cut(df_modelagem['POP_RESIDENTE_MUNIC'], bins=faixas, labels=rotulos, right=False)

In [ ]:
# Distribuição dos indivíduos em cada grupo de município
df_modelagem['porte_municipio'].value_counts()

porte_municipio
Metrópole     399255
Grande        373777
Pequeno II     97796
Médio          84925
Pequeno I      67922
Name: count, dtype: int64

## Segmentação Regiões Brasil

In [72]:
sudeste = [31, 32, 33, 35]
norte = [11,12,13,14,15,16,17]
nordeste = [21,22,23,24,25,26,27,28,29]
centro_oeste = [50, 51, 52, 53]
sul = [41, 42, 43]

# Filtrar para o estado de São Paulo (SG_UF == 35)
df_sudeste = df_modelagem[df_modelagem['SG_UF'].isin(sudeste)].copy()
df_norte = df_modelagem[df_modelagem['SG_UF'].isin(norte)].copy()
df_nordeste = df_modelagem[df_modelagem['SG_UF'].isin(nordeste)].copy()
df_centro_oeste = df_modelagem[df_modelagem['SG_UF'].isin(centro_oeste)].copy()
df_sul =df_modelagem[df_modelagem['SG_UF'].isin(sul)].copy()

# Confirmar o tamanho do novo DataFrame
print(f"Quantidade de registros para Sudeste: {df_sudeste.shape[0]}")
print(f"Quantidade de registros para Norte: {df_norte.shape[0]}")
print(f"Quantidade de registros para Nordeste: {df_nordeste.shape[0]}")
print(f"Quantidade de registros para Centro Oeste: {df_centro_oeste.shape[0]}")
print(f"Quantidade de registros para Sul: {df_sul.shape[0]}")

for df_for in [df_sudeste, df_norte, df_nordeste,df_centro_oeste,df_sul]:
    df_for.drop(columns='SG_UF')

Quantidade de registros para Sudeste: 459163
Quantidade de registros para Norte: 118875
Quantidade de registros para Nordeste: 270175
Quantidade de registros para Centro Oeste: 50278
Quantidade de registros para Sul: 127200


In [73]:
#Proporção de desfecho
pd.DataFrame({'sudeste':df_sudeste['ABANDONO'].value_counts()/df_sudeste.shape[0],
             'norte':df_norte['ABANDONO'].value_counts()/df_norte.shape[0],
             'nordeste':df_nordeste['ABANDONO'].value_counts()/df_nordeste.shape[0],
             'centro_oeste':df_centro_oeste['ABANDONO'].value_counts()/df_centro_oeste.shape[0],
             'sul':df_sul['ABANDONO'].value_counts()/df_sul.shape[0],
})

,sudeste,norte,nordeste,centro_oeste,sul
ABANDONO,,,,,
0,0.848733,0.856564,0.87765,0.859282,0.836313
1,0.151267,0.143436,0.12235,0.140718,0.163687


In [74]:
# 2. Separar variável dependente (y) e independentes (X)
X_sudeste = df_sudeste[variaveis_independentes+['ID_MUNIC_A']]
X_norte = df_norte[variaveis_independentes+['ID_MUNIC_A']]
X_nordeste = df_nordeste[variaveis_independentes+['ID_MUNIC_A']]
X_centro_oeste = df_centro_oeste[variaveis_independentes+['ID_MUNIC_A']]
X_sul = df_sul[variaveis_independentes+['ID_MUNIC_A']]


y_sudeste = df_sudeste['ABANDONO']
y_norte = df_norte['ABANDONO']
y_nordeste = df_nordeste['ABANDONO']
y_centro_oeste = df_centro_oeste['ABANDONO']
y_sul = df_sul['ABANDONO']


#### Subamostragem <p>
Devido a grande quantidade de linhas, iremos subamostrar para rodar os testes

In [75]:
X_sample_sudeste, _, y_sample_sudeste, _ = train_test_split(X_sudeste, y_sudeste, train_size=0.1, stratify=y_sudeste, random_state=42)
X_sample_norte, _, y_sample_norte, _ = train_test_split(X_norte, y_norte, train_size=0.1, stratify=y_norte, random_state=42)
X_sample_nordeste, _, y_sample_nordeste, _ = train_test_split(X_nordeste, y_nordeste, train_size=0.1, stratify=y_nordeste, random_state=42)
X_sample_centro_oeste, _, y_sample_centro_oeste, _ = train_test_split(X_centro_oeste, y_centro_oeste, train_size=0.1, stratify=y_centro_oeste, random_state=42)
X_sample_sul, _, y_sample_sul, _ = train_test_split(X_sul, y_sul, train_size=0.1, stratify=y_sul, random_state=42)

print(f"Quantidade de registros para Sudeste: {X_sample_sudeste.shape[0]}")
print(f"Quantidade de registros para Norte: {X_sample_norte.shape[0]}")
print(f"Quantidade de registros para Nordeste: {X_sample_nordeste.shape[0]}")
print(f"Quantidade de registros para Centro Oeste: {X_sample_centro_oeste.shape[0]}")
print(f"Quantidade de registros para Sul: {X_sample_sul.shape[0]}")

Quantidade de registros para Sudeste: 45916
Quantidade de registros para Norte: 11887
Quantidade de registros para Nordeste: 27017
Quantidade de registros para Centro Oeste: 5027
Quantidade de registros para Sul: 12720


In [76]:
#salvando as amostras
os.makedirs("dados/amostras", exist_ok=True)

amostras = {
    "sudeste": {
        "X_sample": X_sample_sudeste,
        "y_sample": y_sample_sudeste
    },
    "norte": {
        "X_sample": X_sample_norte,
        "y_sample": y_sample_norte
    },
    "nordeste": {
        "X_sample": X_sample_nordeste,
        "y_sample": y_sample_nordeste
    },
    "centro_oeste": {
        "X_sample": X_sample_centro_oeste,
        "y_sample": y_sample_centro_oeste
    },
    "sul": {
        "X_sample": X_sample_sul,
        "y_sample": y_sample_sul
    }
}

# Loop para salvar todos os DataFrames
for regiao, conjuntos in amostras.items():
    for tipo, df in conjuntos.items():
        nome_arquivo = f"dados/amostras/{tipo}_{regiao}.csv"
        df.to_csv(nome_arquivo, index=False)

### Separando Treino e Teste

In [77]:
X_train_sudeste, X_test_sudeste, y_train_sudeste, y_test_sudeste = train_test_split(X_sample_sudeste, y_sample_sudeste, test_size=0.3, random_state=42, stratify=y_sample_sudeste)
X_train_norte, X_test_norte, y_train_norte, y_test_norte = train_test_split(X_sample_norte, y_sample_norte, test_size=0.3, random_state=42, stratify=y_sample_norte)
X_train_nordeste, X_test_nordeste, y_train_nordeste, y_test_nordeste = train_test_split(X_sample_nordeste, y_sample_nordeste, test_size=0.3, random_state=42, stratify=y_sample_nordeste)
X_train_centro_oeste, X_test_centro_oeste, y_train_centro_oeste, y_test_centro_oeste = train_test_split(X_sample_centro_oeste, y_sample_centro_oeste, test_size=0.3, random_state=42, stratify=y_sample_centro_oeste)
X_train_sul, X_test_sul, y_train_sul, y_test_sul = train_test_split(X_sample_sul, y_sample_sul, test_size=0.3, random_state=42, stratify=y_sample_sul)


In [78]:
# Garante que a pasta 'train_test' exista
os.makedirs("dados/train_test", exist_ok=True)

#salvando os csv
dados = {
    "sudeste": {
        "X_train": X_train_sudeste,
        "X_test": X_test_sudeste,
        "y_train": y_train_sudeste,
        "y_test": y_test_sudeste
    },
    "norte": {
        "X_train": X_train_norte,
        "X_test": X_test_norte,
        "y_train": y_train_norte,
        "y_test": y_test_norte
    },
    "nordeste": {
        "X_train": X_train_nordeste,
        "X_test": X_test_nordeste,
        "y_train": y_train_nordeste,
        "y_test": y_test_nordeste
    },
    "centro_oeste": {
        "X_train": X_train_centro_oeste,
        "X_test": X_test_centro_oeste,
        "y_train": y_train_centro_oeste,
        "y_test": y_test_centro_oeste
    },
    "sul": {
        "X_train": X_train_sul,
        "X_test": X_test_sul,
        "y_train": y_train_sul,
        "y_test": y_test_sul
    }
}

# Loop para salvar todos os DataFrames
for regiao, conjuntos in dados.items():
    for tipo, df in conjuntos.items():
        nome_arquivo = f"dados/train_test/{tipo}_{regiao}.csv"
        df.to_csv(nome_arquivo, index=False)